In [ ]:
import numpy as np
#import matplotlib.pyplot as plt
import os
import scipy.io as sio
import numpy as np 

In [ ]:
DATADIR = "./../image_data"
EEG_dir = "./../ImageNet-EEG"

CATEGORIES = ["animal", "blue_sky", "device","flying_object", "green_color", "musical_instrument","plant", "red_color", "vehicle","water", ]
subjects_EEG = ["1","2","3","4","5","6"] 

training_data = []
test_data = []



for category in CATEGORIES:  # do blue_sky and green_color
    path = os.path.join(DATADIR,category)  # create blue_sky and green_color
    class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=blue_sky 1=green_color
    img_counter = 0
    
    for img in os.listdir(path):  # iterate over each image blue_sky and green_color
        # print(img)
        img_counter +=1
        for subject in subjects_EEG: # do 1 ,2, 3, 4, 5, 6
            path_EEG = os.path.join(EEG_dir,subject)
            
            for eeg in os.listdir(path_EEG):
                if img == eeg.strip('.eeg.mat'):
                    if img_counter  < (np.array(os.listdir(path)).size*80/100): # 80 % training 
                        EEGs= sio.loadmat(os.path.join(path_EEG,eeg), matlab_compatible=True)
                        eeg_array = EEGs['x']
                        new_array = eeg_array[319:479,:]
                        training_data.append([new_array, class_num])  # add this to our training_date
                        # print("train"+eeg)
                    else:
                        EEGs= sio.loadmat(os.path.join(path_EEG,eeg), matlab_compatible=True)
                        eeg_array = EEGs['x']
                        new_array = eeg_array[319:479,:]
                        test_data.append([new_array, class_num])  # add this to our training_date
                        # print("test"+eeg)

                            
                                   

In [ ]:
import random

random.shuffle(training_data)
random.shuffle(test_data)


In [ ]:
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
for sample in test_data[:10]:
    print(sample[1])

In [ ]:
X_train = []
y_train = []
for features,label in training_data:
    X_train.append(features)
    y_train.append(label)

X_test = []
y_test = []
for features,label in test_data:
    X_test.append(features)
    y_test.append(label)

In [ ]:
X_train = np.array(X_train).reshape(-1, 160, 128)
X_train.shape

In [ ]:
X_test = np.array(X_test).reshape(-1, 160, 128)
X_test.shape

In [ ]:
from csv import reader
map_dict = dict() # dictionary to hold the target position of a channel in the target 2d map
# read csv file as a list of lists
with open('map_v2.csv', 'r',encoding='utf-8') as csv_file:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(csv_file)
    # Pass reader object to list() to get a list of lists
    m = list(csv_reader)
    print(m)
    for i,row in enumerate(m):
        for j,c in enumerate(row):
            # if the cell is not empty
            if c:
                c = c.replace(u'\ufeff','').strip() # handle encoding issue, to be enhanced
                assert(c not in map_dict)
                map_dict[c] = (i,j) # to be checked as it depends on the orientation
assert(len(map_dict)==128)
print(map_dict)


In [ ]:
map_dict['FFT9h']

In [ ]:
# read csv file as a list of lists
with open('key.csv', 'r',encoding='utf-8') as csv_file:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(csv_file)
    # Pass reader object to list() to get a list of lists
    key = list(csv_reader)
    key =[item for sublist in key for item in sublist] # flatten the key
    assert(len(key)==128)
    print(key)

In [ ]:
#check if all channels are mapped
for k in key:
    if k not in map_dict:
        print(k)

In [ ]:
def vector_to_map(vector):
    #print(vector.shape)
    #print(vector)
    assert(vector.shape==(128,))
    m = np.zeros((14,11))
    for idx, value in np.ndenumerate(vector):
        assert(key[idx[0]] in map_dict) # np return index as tuple 
        x,y = map_dict[key[idx[0]]]
        m[x,y] = value
        #print(m[x,y])
    assert(np.count_nonzero(m)==128)
    return m

In [ ]:
X_train_encoded = np.zeros((X_train.shape[0],X_train.shape[1], 14, 11))
X_test_encoded = np.zeros((X_test.shape[0],X_test.shape[1], 14, 11))

In [ ]:
vector_to_map(X_train[0,0])

In [ ]:
# can this code be enhanced? 
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        X_train_encoded[i,j] = vector_to_map(X_train[i,j])
        
for i in range(X_test.shape[0]):
    for j in range(X_test.shape[1]):
        X_test_encoded[i,j] = vector_to_map(X_test[i,j])

In [ ]:
print( X_train_encoded.shape)
print( X_test_encoded.shape)

In [ ]:
import pickle

pickle_out = open("X_train_encoded_full_2D.pickle","wb")
pickle.dump(X_train_encoded, pickle_out,  protocol=4)
pickle_out.close()

pickle_out = open("y_train_full.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

# We can always load it in to our current script, or a totally new one by doing:

''' 
pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

'''

pickle_out = open("X_test_encoded_full_2D.pickle","wb")
pickle.dump(X_test_encoded, pickle_out, protocol=4)
pickle_out.close()

pickle_out = open("y_test_full.pickle","wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()

In [ ]:
X_train_encoded[1,1,5:10,4:10]